In [5]:
import cv2
import numpy as np
import os
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import os

ModuleNotFoundError: No module named 'cv2'

In [ ]:


# Initialize MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to perform holistic-detection using MediaPipe
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Function to draw styled landmarks on the image
def draw_styled_landmarks(image, results):
    drawing_spec_face = mp.solutions.drawing_utils.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1)
    drawing_spec_pose = mp.solutions.drawing_utils.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1)
    drawing_spec_hands = mp.solutions.drawing_utils.DrawingSpec(color=(121, 22, 76), thickness=1, circle_radius=1)

    mp.solutions.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.solutions.holistic.FACEMESH_TESSELATION, drawing_spec_face)
    mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS, drawing_spec_pose)
    mp.solutions.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS, drawing_spec_hands)
    mp.solutions.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS, drawing_spec_hands)

# Function to extract keypoints from the results
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

# Function to visualize probabilities on the image
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

# Load model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Assuming you have 10 classes (actions)

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'Father','Mother','Yes','No','Help','A','B','C'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
model.load_weights('action.h')

# Define colors for visualization

colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255), (128, 128, 128)]





In [ ]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
Mother
1/1 [==============================] - 0s 16ms/step
Mother
1/1 [==============================] - 0s 32ms/step
Yes
1/1 [==============================] - 0s 35ms/step
Yes
1/1 [==============================] - 0s 32ms/step
Mother
1/1 [==============================] - 0s 24ms/step
Mother
1/1 [==============================] - 0s 32ms/step
C
1/1 [==============================] - 0s 40ms/step
Mother
1/1 [==============================] - 0s 29ms/step
Yes
1/1 [==============================] - 0s 47ms/step
Yes
1/1 [==============================] - 0s 36ms/step
Yes
1/1 [==============================] - 0s 57ms/step
Yes
1/1 [==============================] - 0s 47ms/step
Mother
1/1 [==============================] - 0s 35ms/step
Yes
1/1 [==============================] - 0s 33ms/step
Yes
1/1 [==============================] - 0s 43ms/step
Mother
1/1 [==============================] - 0s 29ms/step
A
1/1 [============================

1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 29ms/step
Father
1/1 [==============================] - 0s 33ms/step
Father
1/1 [==============================] - 0s 29ms/step
Father
1/1 [==============================] - 0s 26ms/step
Father
1/1 [==============================] - 0s 30ms/step
Father
1/1 [==============================] - 0s 34ms/step
Father
1/1 [==============================] - 0s 39ms/step
Father
1/1 [==============================] - 0s 62ms/step
Father
1/1 [==============================] - 0s 36ms/step
Father
1/1 [==============================] - 0s 31ms/step
Father
1/1 [==============================] - 0s 39ms/step
Father
1/1 [==============================] - 0s 33ms/step
Father
1/1 [==============================] - 0s 35ms/step
Father
1/1 [==============================] - 0s 25ms/step
Father
1/1 [==============================] - 0s 26ms/step
Father
1/1 [==============================] - 0s 41ms/step
Fathe

1/1 [==============================] - 0s 28ms/step
Mother
1/1 [==============================] - 0s 30ms/step
B
1/1 [==============================] - 0s 30ms/step
Mother
1/1 [==============================] - 0s 28ms/step
Mother
1/1 [==============================] - 0s 29ms/step
Mother
1/1 [==============================] - 0s 39ms/step
Mother
1/1 [==============================] - 0s 25ms/step
B
1/1 [==============================] - 0s 25ms/step
C
1/1 [==============================] - 0s 59ms/step
Mother
1/1 [==============================] - 0s 32ms/step
Mother
1/1 [==============================] - 0s 40ms/step
Mother
1/1 [==============================] - 0s 26ms/step
Mother
1/1 [==============================] - 0s 24ms/step
Mother
1/1 [==============================] - 0s 29ms/step
Mother
1/1 [==============================] - 0s 24ms/step
Mother
1/1 [==============================] - 0s 27ms/step
Mother
1/1 [==============================] - 0s 25ms/step
Mother
1/1 [=======